In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

plt.style.use("default")
plt.rc("text", usetex=True)
plt.rc("font", family="cm")
plt.rcParams["grid.color"] = (0.5, 0.5, 0.5, 0.2)

In [ ]:
X_dataframe = pd.read_csv("/mnt/ferracci/features_dataframe_new.csv.gz")
X = np.load("/mnt/ferracci/features_new.npz", allow_pickle=True)['a']
y = np.array(pd.read_csv("/mnt/ferracci/targets_dataframe_new.csv.gz")["Qedep"])

In [ ]:
import xgboost as xgb 

dm_train = xgb.DMatrix(X, label=y)

params = {"max_depth": 8, 
          "min_child_weight": 1, 
          "eta": 0.08, 
          "subsample": 0.8, 
          "colsample_bytree": 1, 
          "objective": "reg:squarederror", 
          "eval_metric": "mape",
          "tree_method": "gpu_hist",
}

xgb_cv = xgb.cv(params, dtrain=dm_train, num_boost_round=1000, nfold=5, verbose_eval=False,
                early_stopping_rounds=5, as_pandas=True)

mape_all_features = xgb_cv["test-mape-mean"].min()
std_mape_all_features = xgb_cv.loc[xgb_cv["test-mape-mean"].idxmin(), "test-mape-std"]
print(f"({100 * mape_all_features:.4f} +/- {100 * std_mape_all_features:.4f})%")

In [ ]:
# since the features are highly correlated, not all of them are necessary
def feature_selection(features_dataframe, mape_all_features, std_mape_all_features):
    """
    Adds one by one the feature that maximizes stepwise perfomance gain, until the mape is comparable (inside one
    standard deviation) of the mape obtained using all features.

    Parameters:
        features_dataframe (pd.DataFrame): pandas dataframe with columns corresponding to the computed features
        mape_all_features (float): mape obtained using all features
        std_mape_all_features (float): standard deviation on the mape obtained using all features

    Returns:
        selected_features_names (list): list of strings representing the names of selected features
        trailing_best_mape (list): best mape obtained at each iteration of feature selection
        trailing_std_best_mape (list): standard deviation of best mape for each iteration of feature selection
    """
    selected_features = np.empty((len(features_dataframe), 0)) # empty column vector
    selected_features_names = []
    trailing_best_mape = []
    trailing_std_best_mape = []

    # first iteration: we need to find the feature that provides the best performance on its own
    best_mape = 1 
    for name, feature in features_dataframe.items():
        feature = np.array(feature).reshape(-1, 1)
        # add feature to the stack
        selected_features = np.column_stack((selected_features, feature))
        dm_train = xgb.DMatrix(selected_features, label=y)
        xgb_cv = xgb.cv(params, dtrain=dm_train, num_boost_round=1000, nfold=5, verbose_eval=False,
                    early_stopping_rounds=5, as_pandas=True)
        mape = xgb_cv["test-mape-mean"].min()
        if (mape < best_mape):
            best_mape = mape
            std_mape = xgb_cv.loc[xgb_cv["test-mape-mean"].idxmin(), "test-mape-std"]
            best_feature = feature
            best_feature_name = name
        # pop back feature after loop
        selected_features = np.delete(selected_features, -1, axis=1)

    selected_features = np.column_stack((selected_features, best_feature))
    selected_features_names.append(best_feature_name)
    trailing_best_mape.append(best_mape)
    trailing_std_best_mape.append(std_mape)
    
    # print first selected feature and trailing mape:
    print(f"Adding feature: {selected_features_names[0]} | " +
          f"MAPE: ({100 * trailing_best_mape[0]:.2f} +/- {100 * trailing_std_best_mape[0]:.2f})%") 

    # successive iterations: we need to find the feature that provides the best stepwise performance gain
    while np.abs(trailing_best_mape[-1] - mape_all_features) > std_mape_all_features:
        epsilon = 1e-6 # tolerance on performace gain
        for name, feature in features_dataframe.drop(selected_features_names, axis=1).items():
            feature = np.array(feature).reshape(-1, 1)
            # add feature to the stack
            selected_features = np.column_stack((selected_features, feature))
            dm_train = xgb.DMatrix(selected_features, label=y)
            xgb_cv = xgb.cv(params, dtrain=dm_train, num_boost_round=1000, nfold=5, verbose_eval=False,
                        early_stopping_rounds=5, as_pandas=True)
            mape = xgb_cv["test-mape-mean"].min()
            if (trailing_best_mape[-1] - mape) > epsilon:
                epsilon = trailing_best_mape[-1] - mape # updated highest performange gain
                best_mape = mape
                std_mape = xgb_cv.loc[xgb_cv["test-mape-mean"].idxmin(), "test-mape-std"]
                best_feature = feature
                best_feature_name = name
            # pop back feature after loop
            selected_features = np.delete(selected_features, -1, axis=1)

        selected_features = np.column_stack((selected_features, best_feature))
        selected_features_names.append(best_feature_name)
        trailing_best_mape.append(best_mape)
        trailing_std_best_mape.append(std_mape)

        # print selected feature and trailing mape:
        print(f"Adding feature: {selected_features_names[-1]} | " +
              f"MAPE: ({100 * trailing_best_mape[-1]:.4f} +/- {100 * trailing_std_best_mape[-1]:.4f})%") 

    return selected_features_names, trailing_best_mape, trailing_std_best_mape

In [ ]:
selected_features_names, trailing_best_mape, trailing_std_best_mape = feature_selection(X_dataframe,
                                                                                        mape_all_features,
                                                                                        std_mape_all_features)

with open('/home/ferracci/new_dataset/features_list.txt', 'w') as f:
    f.write(str(selected_features_names))
    f.write('\n')
    f.write(str(trailing_best_mape))
    f.write('\n')
    f.write(str(trailing_std_best_mape))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(11,4), width_ratios=(1,2), dpi=150)

with open('/home/ferracci/new_dataset/features_list.txt', 'r') as f:
    file_content = f.read()

features_list = file_content.split('\n')
selected_features_names = eval(features_list[0])
trailing_best_mape = eval(features_list[1])
trailing_std_best_mape = eval(features_list[2])

ax = axes[0]
ax.errorbar([0, 1, 2, 3], np.array(trailing_best_mape[:4])*100, yerr=np.array(trailing_std_best_mape[:4])*100,
            marker="o", markersize=3, capsize=5., color="k")
ax.axhline(mape_all_features*100, linewidth=1., linestyle="--", dashes=(5, 5))
ax.fill_between(x=range(-1, len(selected_features_names)+1), y1=(mape_all_features-std_mape_all_features)*100, 
                y2=(mape_all_features+std_mape_all_features)*100, alpha=0.3)

ax.set_xlim([-0.2, 3.2])

ax.set_xticks([0, 1, 2, 3])
ax.tick_params(axis="both", which="major", labelsize=12)
ax.tick_params(axis="both", which='minor', labelsize=12)

ax.set_xticklabels(["AccumCharge", "$pe_{std}$", "$\\rho_{cc}$", "$ht_{95\%-90\%}$"], 
                   rotation=-45, ha="left", rotation_mode="anchor")

ax.grid()

ax = axes[1]
ax.errorbar(selected_features_names[3:], np.array(trailing_best_mape[3:])*100, yerr=np.array(trailing_std_best_mape[3:])*100,
            marker="o", markersize=3, capsize=3., color="k")
ax.axhline(mape_all_features*100, linewidth=1., linestyle="--", dashes=(5, 5))
ax.fill_between(x=range(-1, len(selected_features_names[2:])+1), y1=(mape_all_features-std_mape_all_features)*100, 
                y2=(mape_all_features+std_mape_all_features)*100, alpha=0.3)

ax.set_xlim([-0.2, len(selected_features_names[3:])-0.8])

ax.tick_params(axis="both", which="major", labelsize=12)
ax.tick_params(axis="both", which='minor', labelsize=12)

ax.set_xticklabels(["$ht_{95\%-90\%}$", "nPMTs", "$z_{cc}$", "$ht_{entropy}$", "$ht_{5\%-2\%}$", "$ht_{kurtosis}$", "$pe_{mean}$", 
                    "$ht_{10\%-5\%}$", "$pe_{15\%}$", "$ht_{80\%-75\%}$"], 
                   rotation=-45, ha="left", rotation_mode="anchor")

ax.grid()

fig.supxlabel("Added feature", fontsize=15, y=-0.15)
fig.supylabel("MAPE, \%", fontsize=15, x=0.07)

fig.savefig("/home/ferracci/new_dataset/images/BDT_feature_selection.png", dpi=300, bbox_inches="tight", pad_inches=0.2);